In [12]:
import nltk
import pandas as pd
import numpy as np
from lyricsgenius import Genius
from nltk.corpus import wordnet
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence
from langdetect import detect

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
client_access_token = "Z3u3rLvxxvbdSNOItyy-u0IZ60q-5JQqMj_sVIo2tyggpxAk3xPBGSBJMwKlQEeI"

[nltk_data] Downloading package wordnet to /home/plasma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/plasma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/plasma/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [13]:
df = pd.read_csv('muse_v3.csv').astype({'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})
df = df.sample(frac=1).reset_index(drop=True) #randomize row order
df['lyrics'] = ""
df['lemma_lyrics'] = ""
df['textblob_sent'] = 0.0
df['textblob_sent_lemma'] = 0.0
df['vader_sent'] = 0.0
df['vader_sent_lemma'] = 0.0
df['flair_sent'] = 0.0
df['flair_sent_lemma'] = 0.0
df[:10]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
0,https://www.last.fm/music/boa/_/candle%2blights,Candle Lights,Boa,['wintry'],2,3.805000,2.560000,3.225000,a283ac18-28a1-4279-b2c6-735c933beb40,5SsyZgaf9sRyJj0kRnaxqH,j-pop,,,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.last.fm/music/cavalera%2bconspirac...,Sanctuary,Cavalera Conspiracy,['aggressive'],4,4.666667,6.046667,5.596667,66304a47-a43a-4bb9-bd13-863f9baffcba,1qVRv7ydm4sMj9aX5VYuN4,thrash metal,,,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.last.fm/music/remy%2bzero/_/fair,Fair,Remy Zero,"['melancholy', 'sad']",16,6.096560,3.536561,6.008790,8e7a0c99-6c03-4af0-90a0-595787d9b39a,6BGgrZYXAYXiRDBdW1yl2O,indie,,,0.0,0.0,0.0,0.0,0.0,0.0
3,https://www.last.fm/music/father%2bjohn%2bmist...,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,['epic'],3,7.320000,4.837500,5.877500,6a2c1aff-d480-472a-87df-aa35837129c9,2eg2gvPXuwZ9FyrPaLgrXi,folk,,,0.0,0.0,0.0,0.0,0.0,0.0
4,https://www.last.fm/music/vincent%2bdelerm/_/j...,Je t'ai même pas dit,Vincent Delerm,['light'],2,7.275000,4.440000,5.905000,6d548947-747c-4bad-91d0-79f43de32095,65iAmZCnbDS5zQP2Up8vnU,chanson,,,0.0,0.0,0.0,0.0,0.0,0.0
5,https://www.last.fm/music/enigma/_/the%2beyes%...,The Eyes Of Truth (Radio Edit),Enigma,"['meditative', 'enigmatic']",5,3.032500,2.057500,2.512500,9202b8aa-a77d-48dd-a4f8-4c9ece802122,1AhBIWVEQfJP7dliRdYsyA,new age,,,0.0,0.0,0.0,0.0,0.0,0.0
6,https://www.last.fm/music/the%2bmayfield%2bfou...,Eden (Turn The Page),The Mayfield Four,"['powerful', 'hypnotic']",5,6.806000,4.912000,6.046000,NaN,4uk4W1m7xsRj7RxQBVlHj4,alternative rock,,,0.0,0.0,0.0,0.0,0.0,0.0
7,https://www.last.fm/music/m%25e1%25bb%25b9%2bt...,Một Phút Cô Đơn,Mỹ Tâm,['lonely'],1,2.670000,4.370000,3.330000,NaN,2IBBBVgLcLPXSemvkFkl9I,NaN,,,0.0,0.0,0.0,0.0,0.0,0.0
8,https://www.last.fm/music/christine%2band%2bth...,Wandering Lovers,Christine and the Queens,['lush'],1,6.480000,4.240000,5.790000,e2551643-2c0a-4eb9-8cbb-8987db48a3ea,7h1MnX7EqVXmsMgJvwnnP1,electronic,,,0.0,0.0,0.0,0.0,0.0,0.0
9,https://www.last.fm/music/goddess/_/sexual,Sexual,Goddess,"['defiant', 'visceral', 'feral', 'ironic', 'my...",18,5.628261,4.715109,5.394565,a68c2f7f-31f3-45a5-9a1c-680743fa3874,3xfkVkdK3TN8CEdcdc7xf7,electronic,,,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
all_genres = set(df['genre'].unique())
'Rap' in all_genres

False

In [4]:
print("'")

'


In [5]:
sent = TextBlob("adshkbd ahjdba wdjh daih adhas djhad ajhsd asjdh asd aisdh asjhd ").sentiment
print(sent.polarity)


0.0


In [6]:
vader_analyzer = SentimentIntensityAnalyzer()
vader_analyzer.polarity_scores("I hate dogs so much")['compound']

-0.5719

In [7]:
genres = set()
for genre in df['genre']:
    genres.add(genre)
print(genres)

{nan, 'video game music', 'taiko', 'kora', 'swedish metal', 'french', 'fast melodic punk', 'environmental', 'synth-pop', 'glitch', 'native american', 'lute', 'jazz rock', 'death industrial', 'guitar', 'atmospheric black metal', 'field recording', 'pop rap', 'ragtime', 'jazz guitar', 'marching band', 'ragga jungle', 'symphonic metal', 'garage rock revival', 'gnawa', 'stoner metal', 'soundtrack', 'estonian metal', 'nasheed', 'frenchcore', 'russian indie', 'balafon', 'speedcore', 'contemporary country', 'progressive black metal', 'anti-folk', 'pop', 'turkish pop', 'britpop', 'avant-garde metal', 'power electronics', 'dark folk', 'smooth soul', 'carnaval', 'power noise', 'hammond organ', 'prank', 'bolero', 'contemporary folk', 'party', 'grime', 'avant-rock', 'uk pop', 'outlaw country', 'lovers rock', 'russian pop', 'digital hardcore', 'easy listening', 'funk metal', 'vocal house', 'doo-wop', '8-bit', 'industrial noise', 'enka', 'melodic metalcore', 'byzantine', 'melodic black metal', 'loun

In [14]:
def get_flair_sentiment(text, classifier):
    sentences = nltk.tokenize.sent_tokenize(text)
    sentiment = 0
    for sentence in sentences:
        flair_sentence = Sentence(sentence)
        classifier.predict(flair_sentence)
        dictionary = flair_sentence.to_dict()
        if dictionary['labels'][0]['value'] == 'POSITIVE':
            sentiment += dictionary['labels'][0]['confidence']
        else:
            sentiment += (-1*(dictionary['labels'][0]['confidence']))
    return sentiment/len(sentences)

In [15]:
def tag_to_wordnet(tag):
    if tag[0] == ('J') or tag[0] == 's':
        return wordnet.ADJ
    if tag[0] == ('V'):
        return wordnet.VERB
    if tag[0] == ('N'):
        return wordnet.NOUN
    if tag[0] == ('R'):
        return wordnet.ADV
    return None

In [16]:

def preprocess_lyrics(lyrics, lemmatizer):
    words = nltk.word_tokenize(lyrics)
    tagged_words = nltk.pos_tag(words)
    lemmatized_words = []
    for word,tag in tagged_words:
        if word:
            wordnet_tag = tag_to_wordnet(tag)
            if wordnet_tag:
                lemmatized_words.append(lemmatizer.lemmatize(word,wordnet_tag))
            else:
                lemmatized_words.append(lemmatizer.lemmatize(word))
    return " ".join(lemmatized_words)

In [6]:
print(preprocess_lyrics("I looked into the water and waved at the reflection!",nltk.stem.WordNetLemmatizer()))

I look into the water and wave at the reflection !


In [12]:
artist = df.iloc[0]['artist']
track = df.iloc[0]['track']

In [17]:
genius = Genius(client_access_token)
song = genius.search_song("The real slim Shady","Eminem")
print(song.lyrics)

Searching for "The real slim Shady" by Eminem...
Done.
[Intro]
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here...

[Verse 1]
Y'all act like you never seen a white person before
Jaws all on the floor like Pam like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (Agh!)
It's the return of the "Oh, wait, no way, you're kidding
He didn't just say what I think he did, did he?"
And Dr. Dre said… nothing, you idiots!
Dr. Dre's dead, he's locked in my basement (Ha ha!)
Feminist women love Eminem
"Chicka, chicka, chicka, Slim Shady, I'm sick of him
Look at him, walkin' around, grabbin' his you-know-what
Flippin' the you-know-who," "Yeah, but he's so cute though"
Yeah, I probably got a couple of screws up in my head loose
But no worse than what's goin' on in your parents

In [20]:
#test lemmatization
artist = df.iloc[0]['artist']
track = df.iloc[0]['track']
lyrics = genius.search_song(track,artist).lyrics
print(detect(preprocess_lyrics(lyrics,nltk.stem.WordNetLemmatizer())))

Searching for "Candle Lights" by Boa...
Done.
ja


In [21]:
lemmatizer = nltk.stem.WordNetLemmatizer()
vader_analyzer = SentimentIntensityAnalyzer()
flair_classifier = TextClassifier.load('en-sentiment')

2021-12-17 12:23:03,268 loading file /home/plasma/.flair/models/sentiment-en-mix-distillbert_4.pt


In [21]:

for i in range(0,5000):
    print(i)
    artist = df.iloc[i]['artist']
    track = df.iloc[i]['track']
    lyrics = ""
    try:
        lyrics = genius.search_song(track,artist).lyrics
    except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
        lyrics = None
    if not lyrics or len(lyrics)> 15000 or detect(lyrics)!="en" :
        continue
    processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
    df.loc[i,'lemma_lyrics'] = processed_lyrics
    df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
    #TextBlob
    df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
    df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
    #Vader
    df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
    df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
    #Flair
    df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
    df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

print(df[:4])

0
Searching for "Posed By Models" by Young Marble Giants...
Done.
1
Searching for "small mr man pants (w/ andrew broder and orvar smarason)" by Odd Nosdam...
No results found for: 'small mr man pants (w/ andrew broder and orvar smarason) Odd Nosdam'
2
Searching for "Introduction" by Nick Drake...
Specified song does not contain lyrics. Rejecting.
3
Searching for "O Holy Jesus" by The Cathedral Singers...
Done.
4
Searching for "Ушла" by Аукцыон...
Done.
5
Searching for "Melodica (original mix)" by Leama...
Done.
6
Searching for "Laura" by Oscar Peterson...
Done.
7
Searching for "10 - Naive (Big Weekend)" by The Kooks...
No results found for: '10 - Naive (Big Weekend) The Kooks'
8
Searching for "Stay the Night" by James Blunt...
Done.
9
Searching for "The Gambler" by fun....
Done.
10
Searching for "Widdlecombe Fair (Caister Norfolk)" by Tom Brown...
No results found for: 'Widdlecombe Fair (Caister Norfolk) Tom Brown'
11
Searching for "Forever and Ever" by Project Divinity...
Done.
12
Sea

KeyboardInterrupt: 

In [127]:
text = "As Twitter offers a fertile ground for expressing different thoughts and opinions, it can be seen as a valuable tool for sentiment analysis. Furthermore, properly identified reviews present a baseline of information as an input to different systems, such as e-learning systems, decision support systems etc."
processed_text = preprocess_lyrics(text,lemmatizer)
sentences = nltk.tokenize.sent_tokenize(text)
processed_sentences = nltk.tokenize.sent_tokenize(processed_text)
print(processed_sentences)

['As Twitter offer a fertile ground for express different thought and opinion , it can be see a a valuable tool for sentiment analysis .', 'Furthermore , properly identify review present a baseline of information a an input to different system , such a e-learning system , decision support system etc .']


In [168]:
df.loc[0]

lastfm_url                https://www.last.fm/music/eminem/_/%2527till%2...
track                                                      'Till I Collapse
artist                                                               Eminem
seeds                                                        ['aggressive']
number_of_emotion_tags                                                    6
valence_tags                                                           4.55
arousal_tags                                                       5.273125
dominance_tags                                                     5.690625
mbid                                   cab93def-26c5-4fb0-bedd-26ec4c1619e1
spotify_id                                           4xkOaSrkexMciUUogZKVTS
genre                                                                   rap
lyrics                                                                     
lemma_lyrics                                                               
textblob_sen

In [170]:
i = 0
artist = df.iloc[i]['artist']
track = df.iloc[i]['track']
lyrics = ""
try:
    lyrics = genius.search_song(track,artist).lyrics
except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
    lyrics = None
if not lyrics:
    print("No lyrics")
processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
df.loc[i,'lemma_lyrics'] = processed_lyrics
#sentiment analysis part
    
#TextBlob
df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
#Vader
df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
#Flair
df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

Searching for "'Till I Collapse" by Eminem...
Done.


In [30]:
def calculate_sentiment_genres(genres):
    genres = [genre.lower() for genre in genres]
    genre_set= set(genres)
    for i in range(27000,90000):
        if df.iloc[i]['genre'] not in genre_set or df.iloc[i]['lyrics'] != "" or df.iloc[i]['track'] == "":
            continue
        print(i)
        artist = df.iloc[i]['artist']
        track = df.iloc[i]['track']
        lyrics = ""
        try:
            lyrics = genius.search_song(track,artist).lyrics
        except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
            lyrics = None
        if not lyrics or lyrics == "" or len(lyrics)> 15000:
            continue
        processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
        df.loc[i,'lemma_lyrics'] = processed_lyrics
        df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
        #TextBlob
        df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
        df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
        #Vader
        df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
        df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
        #Flair
        df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
        df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

def calculate_sentiment_artists(artists):
    artist_set= set(artists)
    for i in range(0,90000):
        if df.iloc[i]['artist'] not in artist_set or df.iloc[i]['lyrics'] != "":
            continue
        print(i)
        artist = df.iloc[i]['artist']
        track = df.iloc[i]['track']
        lyrics = ""
        try:
            lyrics = genius.search_song(track,artist).lyrics
        except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
            lyrics = None
        if not lyrics or len(lyrics)> 15000 or detect(lyrics)!="en" :
            continue
        processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
        df.loc[i,'lemma_lyrics'] = processed_lyrics
        df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
        #TextBlob
        df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
        df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
        #Vader
        df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
        df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
        #Flair
        df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
        df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

In [31]:
calculate_sentiment_genres(["country", "experimental"])
#"country", "hip-hop","contemporary jazz", "blues", "rock", "experimental", "metal","r&b","folk","rap/"

27034
Searching for "Tiger Eyes (Laid Back)" by Peaking Lights...
Done.
27098
Searching for "Symptoms w/Minstrel Flux" by UniVerse MCs...
No results found for: 'Symptoms w/Minstrel Flux UniVerse MCs'
27109
Searching for "Let's Roll" by Cloud City Rollers...
Done.
27127
Searching for "Цапля" by Dolphin...
No results found for: 'Цапля Dolphin'
27223
Searching for "Two Virgins Side Two" by John Lennon & Yoko Ono...
27276
Searching for "Cruel and unusual." by Chemo Castronovo...
No results found for: 'Cruel and unusual. Chemo Castronovo'
27297
Searching for "VII" by Híbrida...
Done.
27354
Searching for "Mama's Broken Heart" by Miranda Lambert...
Done.
27439
Searching for "The Stars and Stripes Forever" by Matmos...
27476
Searching for "Alisa" by Ariel Pink's Haunted Graffiti...
Done.
27723
Searching for "The Night is Limping" by Mugison...
No results found for: 'The Night is Limping Mugison'
27743
Searching for "No Intention" by Dirty Projectors...
27874
Searching for "Tv Seed" by Farmer J

In [33]:
normalized_df = df.copy()
columns = ['textblob_sent','textblob_sent_lemma','vader_sent','vader_sent_lemma','flair_sent','flair_sent_lemma']
for column in columns:#min max normalization of sentiments
    normalized_df[column] = (df[column] - (-1))/(1-(-1))

normalized_df['valence_tags'] = (df['valence_tags'] - 0)/(10-0)

genre_table = pd.DataFrame(columns=['genre','valence_tags','textblob_sent','textblob_sent_lemma','vader_sent','vader_sent_lemma','flair_sent','flair_sent_lemma'])
genres = ["country", "experimental"]
def average_sentiment_genres(genres,genre_table):#given list of genres, find average sentiment for each one, and return dictionary
    for genre in genres:
        number_songs = len(normalized_df[['valence_tags', 'textblob_sent','textblob_sent_lemma','vader_sent','vader_sent_lemma','flair_sent','flair_sent_lemma']][(df['genre'] == genre) & (normalized_df['lyrics'] != "")])
        dictionary = (normalized_df[['valence_tags', 'textblob_sent','textblob_sent_lemma','vader_sent','vader_sent_lemma','flair_sent','flair_sent_lemma']][(df['genre'] == genre) & (normalized_df['lyrics'] != "")].sum()/number_songs).to_dict()
        dictionary['genre'] = genre
        print(dictionary)
average_sentiment_genres(genres,genre_table)

{'valence_tags': 0.5561097343012972, 'textblob_sent': 0.5538711358259201, 'textblob_sent_lemma': 0.5545968485073915, 'vader_sent': 0.7519771226415094, 'vader_sent_lemma': 0.7586062499999999, 'flair_sent': 0.5965386399708794, 'flair_sent_lemma': 0.6175142734436071, 'genre': 'country'}
{'valence_tags': 0.49882416306880484, 'textblob_sent': 0.5307182818830143, 'textblob_sent_lemma': 0.5324941142914252, 'vader_sent': 0.5770012280701755, 'vader_sent_lemma': 0.5857786842105263, 'flair_sent': 0.5336666790195038, 'flair_sent_lemma': 0.530857355258179, 'genre': 'experimental'}


In [180]:
df['artist'].nunique()

26012

In [181]:
df.iloc[0:10]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,,[ Intro : Eminem ] 'Cause sometimes you just f...,-0.045686,-0.012063,-0.9944,-0.9951,-0.159126,-0.262268
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,,[ Verse ] Saint Anger 'round my neck Saint Ang...,-0.321575,-0.297039,-0.9992,-0.9993,-0.095738,-0.072000
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,,[ Intro : Rick Ross ] Legendary Runners You kn...,0.133909,0.133909,0.9480,0.9737,-0.999637,-0.999840
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,,"[ Intro ] Road runner , road runner Going hund...",-0.062335,-0.065536,0.9701,0.9812,0.562925,0.640141
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771177,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,,Die ! [ Verse ] I do n't need your forgiveness...,-0.481035,-0.481035,-0.9998,-0.9998,-0.702031,-0.641719
5,https://www.last.fm/music/drowning%2bpool/_/st...,Step Up,Drowning Pool,['aggressive'],9,2.971389,5.537500,4.726389,49e7b4d2-3772-4301-ba25-3cc46ceb342e,4Q1w4Ryyi8KNxxaFlOQClK,metal,,"[ Intro ] One , two , three , go ! [ Verse 1 ]...",-0.014527,-0.037016,0.9575,0.9563,0.297424,0.315605
6,https://www.last.fm/music/kanye%2bwest/_/feedback,Feedback,Kanye West,['aggressive'],1,3.080000,5.870000,5.490000,NaN,49fT6owWuknekShh9utsjv,hip-hop,,"[ Chorus ] Ayy , ya hear about the good news ?...",0.141057,0.162426,0.9658,0.9732,-0.112662,0.036590
7,https://www.last.fm/music/deftones/_/7%2bwords,7 Words,Deftones,"['aggressive', 'angry']",10,3.807121,5.473939,4.729091,1a826083-5585-445f-a708-415dc90aa050,6DoXuH326aAYEN8CnlLmhP,nu metal,,"[ Verse 1 ] I 'll never be the same , break de...",-0.188605,-0.179162,-0.9999,-0.9999,-0.528519,-0.451526
8,https://www.last.fm/music/fiona%2bapple/_/limp,Limp,Fiona Apple,"['aggressive', 'angry', 'bitter']",20,3.737211,5.610204,4.626735,4435982c-b83e-4daa-af2b-9f3430036bb7,104YdibC7VQy78xAVmgRYr,singer-songwriter,,"[ Verse 1 ] You wan na make me sick , you wan ...",-0.111548,-0.119589,-0.9969,-0.9853,-0.081627,-0.866962
9,https://www.last.fm/music/metallica/_/sweet%2b...,Sweet Amber,Metallica,['aggressive'],4,3.582759,5.757241,5.340000,fe1cc051-faa7-4953-b331-f6196cd3ddae,5fU6qjmD38P90BMsuqpiuU,metal,,[ Verse 1 ] Wash your back so you do n't stab ...,0.242893,0.231502,0.9973,0.9972,0.787943,0.341907


In [182]:
df.to_csv('partialcsv.csv', index = False)

In [195]:
df.iloc[25:30]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
25,https://www.last.fm/music/a%2bflock%2bof%2bsea...,Standing in the Doorway,A Flock of Seagulls,['exciting'],1,7.950000,6.950000,7.21000,9514ad3b-b995-4ac9-bbe2-0ca0a2973df2,4pkEoiyCaOtwxUL8C6IGkS,new wave,I can see you standing outside\nSee your shado...,I can see you stand outside See your shadow on...,-0.058333,-0.058333,0.6124,0.6124,-0.590647,-0.691504
26,https://www.last.fm/music/tiger%2blou/_/nation...,National Ave,Tiger Lou,['reflective'],3,5.400000,3.060000,6.08000,72e814fd-a2f5-4049-9e36-451811f487f0,3qvA3ZBRYgZ6ci2YItP4Ye,indie,Where do you wanna go?\nI wanna go home\nHome ...,Where do you wan na go ? I wan na go home Home...,0.250000,-0.087500,-0.2500,-0.2500,0.780610,0.751677
27,https://www.last.fm/music/eufonius/_/wish,wish,eufonius,['pure'],1,6.800000,4.050000,6.44000,941100ab-9e8a-4f7d-9218-42aa6f543e0c,NaN,j-pop,CHAPTER TWO\n\nTHE MYSTERIES OF ALBERTINE — TH...,CHAPTER TWO THE MYSTERIES OF ALBERTINE — THE G...,0.101581,0.101506,0.0000,0.0000,0.000000,0.000000
28,https://www.last.fm/music/jon%2bhassell/_/char...,Charm (Over Burundi Cloud),Jon Hassell,['sophisticated'],8,6.163750,3.881250,6.02375,NaN,NaN,soundtrack,,,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000
29,https://www.last.fm/music/hard-fi/_/help%2bme%...,Help Me Please,Hard-Fi,"['romantic', 'soft']",6,5.697227,3.487815,5.83395,NaN,NaN,indie,,,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000


In [32]:
print(all_genres)

{nan, 'video game music', 'taiko', 'kora', 'swedish metal', 'french', 'fast melodic punk', 'environmental', 'synth-pop', 'glitch', 'native american', 'lute', 'jazz rock', 'death industrial', 'guitar', 'atmospheric black metal', 'jazz guitar', 'field recording', 'ragtime', 'pop rap', 'marching band', 'ragga jungle', 'symphonic metal', 'gnawa', 'garage rock revival', 'stoner metal', 'soundtrack', 'estonian metal', 'nasheed', 'russian indie', 'frenchcore', 'balafon', 'speedcore', 'contemporary country', 'progressive black metal', 'anti-folk', 'pop', 'turkish pop', 'britpop', 'avant-garde metal', 'power electronics', 'dark folk', 'smooth soul', 'carnaval', 'power noise', 'hammond organ', 'prank', 'bolero', 'contemporary folk', 'party', 'grime', 'uk pop', 'avant-rock', 'outlaw country', 'lovers rock', 'russian pop', 'digital hardcore', 'easy listening', 'funk metal', 'vocal house', 'doo-wop', '8-bit', 'industrial noise', 'enka', 'melodic metalcore', 'byzantine', 'melodic black metal', 'loun